# 15 STAC specification

So far in our course we have obtained data in two ways: by downloading it directly from the data provider or by obtaining a URL from a data repository. 
This can be a convenient way to access targeted datasets, often usign graphical user interfaces (GUIs) for data discovery and filtering.
However, relying on clicking and copy-pasting addresses and file names can make our workflows more error-prone and less reproducible. 
In particular, satellites around the world produce terabytes of new data daily and manually browsing through data repositories can it make difficult to access this data. 
Moreover, it can be inconvenient to learn a new way to access data from every single big data provider.
This is where STAC comes in.

![](/book/images/lesson-16/STAC-01.png){width="60%" fig-align="center"}

The **SpatioTemporal Asset Catalog (STAC)** is an emerging open standard for geospatial data that aims to increase the interoperability of geospatial data, particularly satellite imagery. 
[Many major data archives](https://stacspec.org/en/about/datasets/) now follow the STAC specification.

In the next classes we'll be working with the [Microsoft's Planetary Computer (MPC)](https://planetarycomputer.microsoft.com) STAC API. 
In this lesson we will learn about the main components of a STAC catalog and how to search for data using the MPC's STAC API. 

## Item, collection, and catalog
The **STAC item** (or just item) is the building block of a STAC. 
An item is a GeoJSON feature with additional fields that make it easier to find it as we look for data across catalogs. 

An item holds two types of information:

1. **Metadata:** The metadata for a STAC item includes core identifying information (such as ID, geometry, bounding box, and date), and additional properties (for example, place of collection). 

2. **Assets:** Assets are links to the actual data of the item (for example, links to the spectral bands of a satellite image.)

STAC items can be grouped into **STAC collections**. 
For example, while a single satellite scene (at a single time and location) would constitue an item, scenes across time and location from the same satellite can be orgnanized in a collection. 
Finally, multiple collections can be organized into a single **STAC catalog**. 

For example, we'll be accessing the MPC STAC catalog. Two of its collections are the National Agriculture Imagery Program (NAIP) colelction and the Copernicus Digital Elevation Model (DEM) colleciton. Each of these collections has multiple items, with item cotaining properties (metadata) and assets (links to the data). 

![](/book//images/lesson-16/mpc-stac-catalog.png)

## Application Programming Interface (API)
To request data from a catalog following the STAC standard we use an **Application Programming Interface (API)**. 
We can think of an API as an intermediary tasked with sending our request for data to the data catalog and getting the response from the catalog back to us. 
The following diagram nicely explains what an API does using a real-life analogy of a restaurant:

![Image Source: [Geeks for geeks - What is an API?](https://www.geeksforgeeks.org/what-is-an-api/)](/book/images/lesson-16/what-is-an-api.png)

The Python package to access APIs for STAC catalogs is [`pystac_client`](https://pystac-client.readthedocs.io/en/stable/). 
Our goal in this lesson is to retrieve [NAIP data](https://naip-usdaonline.hub.arcgis.com) from the [MPC's data catalog](https://planetarycomputer.microsoft.com/catalog) via its STAC API. 

## MPC Catalog 
<!--https://github.com/NCEAS/msai4earth-esa/blob/main/examples/naip_example/access_naip.ipynb-->
First, load the necessary packages:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import rioxarray as rioxr
from shapely.geometry import Polygon

from pystac_client import Client  # To access STAC catalogs

import planetary_computer  # To sign items from the MPC STAC catalog 

from IPython.display import Image  # To nicely display images

### Access
We use the `Client` function from the `pystac_client` package to access the catalog:

In [ ]:
# Access MPC catalog
catalog = Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

The `modifier` parameter is needed to access the data in the MPC catalog.

### Cata log Exploration
Let's check out some of the catalog's metadata:

In [ ]:
# Explore catalog metadata
print('Title:', catalog.title)
print('Description:', catalog.description)

We can access its collections by using the `get_collections()` method:

In [ ]:
catalog.get_collections()

Notice the output of `get_collections()` is a **generator**. 
This is a special kind of **lazy object** in Python over which you can loop over like a list. 
Unlike a list, the items in a generator do not exist in memory until you explicitely iterate over them or convert them to a list. This can allow for more efficient memory allcoation. Once the generator is exhausted (i.e. iterated over completely), it cannot be reused unless it is recreated.

Let's try getting the collections from the catalog again:

In [ ]:
# Get collections and print their names
collections = list(catalog.get_collections())  # Turn generator into list

print('Number of collections:', len(collections))

print("Collections IDs (first 10):")
for i in range(10):
    print('-', collections[i].id)

## Collection
The NAIP catalog's ID is `'naip'`. 
We can select a single collection for exploration using the `get_child()` method for the catalog and the collection ID as the parameter:

In [ ]:
naip_collection = catalog.get_child('naip')
naip_collection

<!--
https://pystac.readthedocs.io/en/latest/api/pystac.html#pystac.Collection

https://pystac.readthedocs.io/en/stable/api/item_collection.html#pystac-item-collection
-->

## Catalog search
We can narrow down the search within the `catalog` by specifying a time range, an area of interest, and the collection name. 
The simplest ways to define the area of interest to look for data in the catalog are:

- a GeoJSON-type dictionary with the coordinates of the bounding box,
- as a list `[xmin, ymin, xmax, ymax]` with the coordinate values defining the four corners of the bounding box.

In this lesson we will look for the NAIP scenes over Santa Barbara from 2018 to 2023. 
We'll use the GeoJSON method to define the area of interest:

In [ ]:
# Temporal range of interest
time_range = "2018-01-01/2023-01-01"

# NCEAS bounding box (as a GeoJSON)
bbox = {
    "type": "Polygon",
    "coordinates":[
        [
            [-119.70608227128903, 34.426300194372274],
            [-119.70608227128903, 34.42041139020533],
            [-119.6967885126002, 34.42041139020533],
            [-119.6967885126002, 34.426300194372274],
            [-119.70608227128903, 34.426300194372274]
        ]
    ],
}

# Catalog search
search = catalog.search(
    collections = ['naip'],
    intersects = bbox,
    datetime = time_range)
search

To get the items found in the search (or check if there were any matches in the search) we use the `item_collection()` method:

In [ ]:
# Retrieve search items
items = search.item_collection()
len(items)

This output tells us there were three items in the catalog that matched our search!

In [ ]:
items

## Item
<!--https://pystac.readthedocs.io/en/stable/api/pystac.html#pystac.Item-->
Let's get the first item in the search:

In [ ]:
# Get first item in the catalog search
item = items[0]
type(item)

Remember the [STAC item](https://pystac.readthedocs.io/en/stable/api/pystac.html#pystac.Item) is the core object in a STAC catalog. 
The item does not contain the data itself, but rather metadata and assets that contain links to access the actual data.
Some of the metadata:

In [ ]:
# Print item ID and properties
print('ID:' , item.id)
item.properties

Just as the item properties, the item assets are given in a dictionary, with each value being a [`pystac.asset`](https://pystac.readthedocs.io/en/stable/api/asset.html)
Let's check the assets in the `item`:

In [ ]:
item.assets

In [ ]:
for key in item.assets.keys():
    print(key, '--', item.assets[key].title)

Notice each asset has an `href`, which is a link to the data. 
For example, we can use the URL for the `'rendered_preview'` asset to plot it:

In [ ]:
# Plot rendered preview
Image(url=item.assets['rendered_preview'].href, width=500)

## Load data
The raster data in our current `item` is in the `image` asset. 
Again, we access this data via its URL. 
This time, we open it using `rioxr.open_rasterio()` directly:

In [ ]:
sb = rioxr.open_rasterio(item.assets['image'].href)
sb

Notice this raster has four bands (red, green, blue, nir), so we cannot use the `.plot.imshow()` method directly (as this function only works when we have three bands). 
Thus we need select the bands we want to plot (RGB) before plotting:

In [ ]:
#| eval: false
# Plot raster with correct ratio
size = 6  
aspect = sb.rio.width / sb.rio.height 
# Select R,G,B bands and plot
sb.sel(band=[1,2,3]).plot.imshow(size=size, aspect=aspect)

![](/book/images/lesson-16/sb-from-stac-item.png){width="70%"}

:::{.callout-tip}
## Exercise
The `'cop-dem-glo-90'` collection contains the Copernicus Digital Elevation Model (DEM) at 90m resolution data. 

1) Reuse the `bbox` for Santa Barbara to look for items in this collection. 
2) Get the first item in the search and examine its assets.
3) Check the item's rendered preview asset by clicking on it's URL.
4) Open and plot the item's data using `rioxarray`.
5) Obtain the maximum and minimum elevation on the scene as *numbers*. 
6) Print the maximum and minimum elevation [rounded to two decimal points using f-strings](https://stackoverflow.com/questions/46062105/rounding-floats-with-f-string). 
:::


<!--
## Exercise
Recreate the following image, where the red rectangle shows the bounding box used to do the catalog search. 
HINT: `bbox` as it is now is a dictionary. How can you get the vertices list out of it to first create a `shapely` `Polygon`?

![](/book/images/lesson-16/sb-and-aoi-mpc.png)


In [ ]:
#| eval: false
#| 
aoi = gpd.GeoDataFrame(geometry=[Polygon(bbox['coordinates'][0])],
                crs='epsg:4326')
aoi.plot()

In [ ]:
#| eval: false
#| 
# reproject aoi to rgb crs
aoi = aoi.to_crs(sb.rio.crs)
print('matched crs?',  aoi.crs == sb.rio.crs)
aoi.crs

In [ ]:
#| eval: false
#| 
fig, ax = plt.subplots()

size = 6 # height in in of plot height
aspect = sb.rio.width / sb.rio.height # ratio of widht/height
fig.set_size_inches((size, size*aspect)) # reuse size and aspect

sb.sel(band=[1,2,3]).plot.imshow(ax=ax)
aoi.plot(ax=ax, color='red', alpha=0.6)

-->


## References
STAC Documentation:

- [The STAC Specification](https://stacspec.org/en/tutorials/intro-to-stac/)

- [Read a STAC Catalog Using PySTAC](https://stacspec.org/en/tutorials/1-read-stac-python/)

[Microsoft Planetary Computer Documentation - Reading Data from the STAC API](https://planetarycomputer.microsoft.com/docs/quickstarts/reading-stac/)

<!-- https://planetarycomputer.microsoft.com/docs/tutorials/hurricane-florence-animation/

Idea for homework:

https://planetarycomputer.microsoft.com/dataset/io-biodiversity#Example-Notebook

-->

<!--
An easy way to obtain coordinates for such a region:

![Screenshot from geojson.io](/images/geojsonio-sb.png)

1. go to  [https://geojson.io/](https://geojson.io/) website,
2. zoom in until you find the NCEAS building in Santa Barbara, it might help to change to 'Satellite Streets' view on the bottom left corner, 
3. click on the rectangle icon on the right-side toolbar and draw a small region around the NCEAS buildng,
4. the geoJSON code representing this area will appear in the code box,
5. one option is two copy-paste this geoJSON into an empty text file and save such file with the .json extension, we could then read it in using `geopandas`
6. instead, we will create our region of index by just *copy-pasting the list of points* and storing it as a variable:

In [ ]:
# vertices of our aoi box
points = [[-119.70608227128903, 34.426300194372274],
          [-119.70608227128903, 34.42041139020533],
          [-119.6967885126002, 34.42041139020533],
          [-119.6967885126002, 34.426300194372274],
          [-119.70608227128903, 34.426300194372274]]

We can then create a new `geopandas.GeoDataFrame`:

In [ ]:
# create geodataframe with aoi 
aoi = gpd.GeoDataFrame(geometry=[Polygon(points)],
                           crs='epsg:4326')
aoi

Let's break this down a bit:

- first, we use the `shapely`'s `Polygon()` function to create a polygon from our `points` list. 
- in `[Polygon(points)]` we put this polygon inside a list so we can form the geometry column of our new `gpd.GeoDataFrame`
- we know all the geoJSON files have CRS equal to EPSG:4326/WGS 84, so we set the the CRS of our new `gpd.GeoDataFrame` to this.
-->